## Problem 3
* Dataset used will be from Course Project, but will perform different question not used in course project
* the first question will be a classical question to see whether I can predict Odor from the predictor variables
* the second question will deal with unsupervised learning and see whether I can learn hidden patterns from the
mushroom categories themselves
* Evaluation and reference to which datacamp course was used will be documented in the notebook after each trial run

In [1]:
# Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score, classification_report
import plotly.express as px

from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
import joblib

from sklearn import set_config
sns.set_palette('Set2')
set_config(display='diagram')

from yellowbrick.cluster import KElbowVisualizer


## First Question:
### Predict the odor of mushroom from its predictor variables (different from course project which predicts edibility)
### Classification methods will be used
### Dataset: Mushroom edibility, source: from https://archive.ics.uci.edu/ml/datasets/mushroom
* imported dataset will be already cleaned from course project

In [2]:
# import dataset
mushroom = pd.read_csv('data/mushroom_imputed_2_adj.csv')
mushroom.head()

,Class,Cap-Shape,Cap-Surface,Cap-Color,Bruises,Odor,Gill-Attachment,Gill-Spacing,Gill-Size,Gill-Color,...,Stalk-Surface-Above-Ring,Stalk-Surface-Below-Ring,Stalk-Color-Above-Ring,Stalk-Color-Below-Ring,Veil-Color,Ring-Number,Ring-Type,Spore-Print-Color,Population,Habitat
0,poisonous,convex,smooth,brown,bruises,pungent,free,close,narrow,black,...,smooth,smooth,white,white,white,one,pendant,black,scattered,urban
1,edible,convex,smooth,yellow,bruises,almond,free,close,broad,black,...,smooth,smooth,white,white,white,one,pendant,brown,numerous,grasses
2,edible,bell,smooth,white,bruises,anise,free,close,broad,brown,...,smooth,smooth,white,white,white,one,pendant,brown,numerous,meadows
3,poisonous,convex,scaly,white,bruises,pungent,free,close,narrow,brown,...,smooth,smooth,white,white,white,one,pendant,black,scattered,urban
4,edible,convex,smooth,gray,no,none,free,crowded,broad,black,...,smooth,smooth,white,white,white,one,evanescent,brown,abundant,grasses


In [54]:
X = mushroom.drop('Odor', axis=1)
y = mushroom[['Odor']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
X_train.shape

(6084, 21)

In [55]:
# Preprocessing the training data

ordinal_X = OrdinalEncoder()
X_train = ordinal_X.fit_transform(X_train)

ordinal_y = OrdinalEncoder()
y_train = ordinal_y.fit_transform((y_train))

display('X train', X_train, 'y train', y_train)


'X train'

array([[1., 4., 2., ..., 7., 4., 1.],
       [1., 4., 3., ..., 7., 4., 6.],
       [0., 2., 2., ..., 0., 4., 6.],
       ...,
       [0., 3., 2., ..., 1., 5., 3.],
       [0., 0., 3., ..., 7., 3., 0.],
       [0., 4., 0., ..., 7., 2., 0.]])

'y train'

array([[3.],
       [3.],
       [6.],
       ...,
       [1.],
       [6.],
       [6.]])

In [56]:
# preprocess the test data

X_test = ordinal_X.transform(X_test)

y_test = ordinal_y.transform(y_test)

display('X test', X_test, 'y test', y_test)

'X test'

array([[0., 3., 0., ..., 1., 4., 6.],
       [0., 2., 0., ..., 1., 4., 6.],
       [0., 2., 3., ..., 1., 4., 1.],
       ...,
       [1., 3., 2., ..., 7., 1., 6.],
       [0., 0., 3., ..., 1., 1., 1.],
       [0., 2., 2., ..., 0., 5., 6.]])

'y test'

array([[6.],
       [1.],
       [6.],
       ...,
       [5.],
       [6.],
       [6.]])

In [57]:
# make y_train and y_test one dimensional for random forest classifier

y_train = np.concatenate(y_train).ravel()
y_test = np.concatenate(y_test).ravel()
y_train, y_test

(array([3., 3., 6., ..., 1., 6., 6.]), array([6., 1., 6., ..., 5., 6., 6.]))

In [7]:
# run data on Decision Tree Classifier

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), {}, cv=5)
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={})

In [8]:
print('train score', rf_grid.best_score_)
print('test score', rf_grid.score(X_test, y_test))
print('std dev', rf_grid.cv_results_['std_test_score'][rf_grid.best_index_])

train score 0.7281393255632919
test score 0.6962524654832347
std dev 0.006159606539859663


This means we are overfitting our dataset and need to perform some hyperparameter tuning to reduce this overfit


In [9]:
# higher estimators reduces variance,
# higher min samples will also reduce overfit
# smaller number of max featuers will reduce overfit

rf_param_grid = {
    'n_estimators':[250, 500],
     'max_features':['log2', 'sqrt'],
     'min_samples_leaf':[175, 200, 500, 1000]
}

In [10]:
# rf_grid2 = GridSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=5)
# rf_grid2.fit(X_train, y_train)
rf_grid2 = RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=5, random_state=42)
rf_grid2.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   param_distributions={'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [175, 200, 500,
                                                             1000],
                                        'n_estimators': [250, 500]},
                   random_state=42)

In [11]:
print('train score', rf_grid2.best_score_)
print('test score', rf_grid2.score(X_test, y_test))
print('std dev', rf_grid2.cv_results_['std_test_score'][rf_grid2.best_index_])

train score 0.7596985414954807
test score 0.7731755424063116
std dev 0.004702713080633196


In [12]:
rf_grid2.best_params_

{'n_estimators': 250, 'min_samples_leaf': 175, 'max_features': 'log2'}

In [59]:
print('Confusion Matrix \n',confusion_matrix(rf_grid2.predict(X_test),y_test), '\n')
print('Accuracy Score \n', rf_grid2.score(X_test, y_test), '\n')
print('Classification Report \n',classification_report(rf_grid2.predict(X_test),y_test))


Confusion Matrix 
 [[ 58  67   0   0   0   0   0   0   0]
 [ 17  10   0   0   0   0   0   0   0]
 [  0   0  50   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0 149 519  11   9   0 134]
 [  0   0   0   0   0   0   0   0   0]
 [ 27  33   0   0   0   0 852   0   0]
 [  0   0   0   0   0   0   0  76   0]
 [  0   0   0   4   9   0   0   0   3]] 

Accuracy Score 
 0.7731755424063116 

Classification Report 
               precision    recall  f1-score   support

         0.0       0.57      0.46      0.51       125
         1.0       0.09      0.37      0.15        27
         2.0       1.00      1.00      1.00        50
         3.0       0.00      0.00      0.00         0
         4.0       0.98      0.63      0.77       822
         5.0       0.00      0.00      0.00         0
         6.0       0.99      0.93      0.96       912
         7.0       1.00      1.00      1.00        76
         8.0       0.02      0.19      0.04        16

    accuracy           

/home/wkc/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wkc/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wkc/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
# to aid in deciphering confusion matrix
for num, odor in enumerate(ordinal_y.categories_[:][0].tolist()):
    print(num, odor)

0 almond
1 anise
2 creosote
3 fishy
4 foul
5 musty
6 none
7 pungent
8 spicy


In [86]:
pd.Series(rf_grid2.best_estimator_.feature_importances_, index=X.columns).sort_values(ascending=False)

Class                       0.240646
Spore-Print-Color           0.122426
Gill-Size                   0.081778
Ring-Type                   0.063429
Stalk-Shape                 0.055286
Cap-Color                   0.052545
Stalk-Color-Above-Ring      0.047043
Stalk-Color-Below-Ring      0.044781
Stalk-Surface-Below-Ring    0.044467
Stalk-Root                  0.044097
Population                  0.035879
Stalk-Surface-Above-Ring    0.035127
Bruises                     0.030531
Gill-Spacing                0.028306
Habitat                     0.024324
Gill-Color                  0.020235
Ring-Number                 0.015766
Cap-Surface                 0.009843
Cap-Shape                   0.003490
Veil-Color                  0.000000
Gill-Attachment             0.000000
dtype: float64

## Evaluation
With hyperaparemter tuning we were better able to predict odors of a mushroom from its features. The tuned forest
yielded a much better accuracy fit with a 1% difference and better performance than the untuned random forest classifier.
Also it did not overfit like the untune forest classifier.

Also from the confusion matrix it appears that our model does a poot job in precision of anise and spicy odors
It also appears that fishy and musty odors are not included in our dataset.
Our model does a very accurate job in precision and recall for fishy, none, and pungent smells. It odes a moderate job
for almond smells.

Overally the accuracy of our model yeilds a score of .77 which is strong score.

Last the features that were regarded as most important in predicting our model were class, which was edibility of mushroom,
spore-print color, Gill-size, Ring-Type, and Stalk-shape

From datacamp, the methods used such as Random Forest Classifer as well as hypertuning, came from the
'Machine Learning with Tree-Based Models in Python' course.

## Second Question
### Use clustering to learn hidden patterns from the mushroom dataset
### Using same dataset as First Question

In [14]:
# # label encode our dataset before using a random forest regressor for it
#
# processor = Pipeline([
#     ('processing', ColumnTransformer([('ordinal', OrdinalEncoder(), ['Gender']),
#                                       ('spending_group_encoder', OrdinalEncoder(categories=[['low', 'medium', 'high']]), ['SpendingGroup']),
#                                       ], remainder='passthrough')),
# ])
#
# processor
#

In [15]:
# X = processor.fit_transform(customer)